# AI Job Market Salary Prediction

## Problem

Predict salary ranges for AI/ML job postings based on job characteristics including skills, tools, location, employment type, job title, industry, and company.

## Solution

Train a Multi-Layer Perceptron (MLP) neural network to classify job postings into salary categories. We compare multiple architectures: a linear baseline, MLP with ReLU activation, and an improved MLP with dropout regularization.


## Data Loading and Preprocessing


In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import torchinfo
import my
from joblib import load


In [2]:
df = pd.read_csv('./ai_job_market.csv')
df


,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,company_size,tools_preferred
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025-08-20,Large,"KDB+, LangChain"
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024-03-22,Large,"FastAPI, KDB+, TensorFlow"
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025-09-18,Large,"BigQuery, PyTorch, Scikit-learn"
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024-05-08,Large,"TensorFlow, BigQuery, MLflow"
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025-02-24,Large,"PyTorch, LangChain"
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,"Mueller, Ellis and Clark",Finance,NLP Engineer,"Flask, FastAPI, Power BI",Senior,Internship,"Washingtonmouth, SD",90382-110126,2024-04-22,Large,MLflow
1996,1997,Roberts-Yu,Automotive,AI Product Manager,"R, Flask, Excel, C++, CUDA, Scikit-learn",Mid,Remote,"Joshuafort, ZA",47848-137195,2023-12-02,Large,"KDB+, LangChain, MLflow"
1997,1998,"Brooks, Williams and Randolph",Education,Data Analyst,"Hugging Face, Excel, Scikit-learn, R, MLflow",Entry,Contract,"West Brittanyburgh, CG",134994-180108,2023-10-29,Large,PyTorch
1998,1999,Castaneda-Smith,Education,Quant Researcher,"AWS, Python, Scikit-learn",Senior,Contract,"Anthonyshire, OM",62388-82539,2024-08-10,Large,"MLflow, TensorFlow, FastAPI"


In [3]:

df['salary_range_usd']
df[['Min_Salary', 'Max_Salary']] = df['salary_range_usd'].str.split('-', expand=True)
df['Min_Salary'] = pd.to_numeric(df['Min_Salary'], errors='coerce')
df['Max_Salary'] = pd.to_numeric(df['Max_Salary'], errors='coerce')
df['Average_Salary']=((df["Min_Salary"]+df["Max_Salary"])/20000)


skills=df["skills_required"].str.split(',', expand=True)
skills.columns = [f'Skill_{i+1}' for i in range(skills.shape[1])]
tools=df["tools_preferred"].str.split(',', expand=True)
tools.columns = [f'Tools_{i+1}' for i in range(tools.shape[1])]
skills
df['Average_Salary']


0       10.12290
1       11.16990
2       17.08500
3        8.73255
4       11.70535
          ...   
1995    10.02540
1996     9.25215
1997    15.75510
1998     7.24635
1999     7.66045
Name: Average_Salary, Length: 2000, dtype: float64

In [4]:

df = pd.concat([df, skills,tools], axis=1)
df


,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,...,Average_Salary,Skill_1,Skill_2,Skill_3,Skill_4,Skill_5,Skill_6,Tools_1,Tools_2,Tools_3
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025-08-20,...,10.12290,NumPy,Reinforcement Learning,PyTorch,Scikit-learn,GCP,FastAPI,KDB+,LangChain,None
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024-03-22,...,11.16990,Scikit-learn,CUDA,SQL,Pandas,None,None,FastAPI,KDB+,TensorFlow
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025-09-18,...,17.08500,MLflow,FastAPI,Azure,PyTorch,SQL,GCP,BigQuery,PyTorch,Scikit-learn
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024-05-08,...,8.73255,Scikit-learn,C++,Pandas,LangChain,AWS,R,TensorFlow,BigQuery,MLflow
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025-02-24,...,11.70535,Excel,Keras,SQL,Hugging Face,None,None,PyTorch,LangChain,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,"Mueller, Ellis and Clark",Finance,NLP Engineer,"Flask, FastAPI, Power BI",Senior,Internship,"Washingtonmouth, SD",90382-110126,2024-04-22,...,10.02540,Flask,FastAPI,Power BI,None,None,None,MLflow,None,None
1996,1997,Roberts-Yu,Automotive,AI Product Manager,"R, Flask, Excel, C++, CUDA, Scikit-learn",Mid,Remote,"Joshuafort, ZA",47848-137195,2023-12-02,...,9.25215,R,Flask,Excel,C++,CUDA,Scikit-learn,KDB+,LangChain,MLflow
1997,1998,"Brooks, Williams and Randolph",Education,Data Analyst,"Hugging Face, Excel, Scikit-learn, R, MLflow",Entry,Contract,"West Brittanyburgh, CG",134994-180108,2023-10-29,...,15.75510,Hugging Face,Excel,Scikit-learn,R,MLflow,None,PyTorch,None,None
1998,1999,Castaneda-Smith,Education,Quant Researcher,"AWS, Python, Scikit-learn",Senior,Contract,"Anthonyshire, OM",62388-82539,2024-08-10,...,7.24635,AWS,Python,Scikit-learn,None,None,None,MLflow,TensorFlow,FastAPI


In [5]:
label1, uniqueValues1= pd.factorize(df['Skill_1'])
label2, uniqueValues2= pd.factorize(df['Skill_2'])
label3, uniqueValues3= pd.factorize(df['Skill_3'])
label4, uniqueValues4= pd.factorize(df['Skill_4'])
label5, uniqueValues5= pd.factorize(df['Skill_5'])
label6, uniqueValues6= pd.factorize(df['Tools_1'])
label7, uniqueValues7= pd.factorize(df['Tools_2'])
label8, uniqueValues8= pd.factorize(df['Tools_3'])
label9, uniqueValues9= pd.factorize(df['location'])
label10, uniqueValues10= pd.factorize(df['employment_type'])
label11, uniqueValues11= pd.factorize(df['job_title'])
label12, uniqueValues12= pd.factorize(df['industry'])
label13, uniqueValues13= pd.factorize(df['company_name'])



In [6]:
targetlabel, targetunique= pd.factorize(df['Average_Salary'])
df['target']=targetlabel
df['target']
targetunique

Index([ 10.1229,  11.1699,   17.085,  8.73255, 11.70535,  13.6675,  7.95845,
         7.2588, 16.13715,  12.2195,
       ...
        16.5924,  11.4291,  5.49335,  19.1729,  10.7884,  10.0254,  9.25215,
        15.7551,  7.24635,  7.66045],
      dtype='float64', length=1992)

In [7]:
df['fskill_1']=label1
df['fskill_2']=label2
df['fskill_3']=label3
df['fskill_4']=label4
df['fskill_5']=label5
df['ftool_1']=label6
df['ftool_2']=label7
df['ftool_3']=label8
df['floc']=label9
df['ftype']=label10
df['ftitle']=label11
df['findustry']=label12
df['fname']=label13


In [8]:
df


,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,...,fskill_4,fskill_5,ftool_1,ftool_2,ftool_3,floc,ftype,ftitle,findustry,fname
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025-08-20,...,0,0,0,0,-1,0,0,0,0,0
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024-03-22,...,1,-1,1,1,0,1,0,1,1,1
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025-09-18,...,2,1,2,2,1,2,0,2,1,2
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024-05-08,...,3,2,3,3,2,3,0,3,1,3
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025-02-24,...,4,-1,4,0,-1,4,1,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,"Mueller, Ellis and Clark",Finance,NLP Engineer,"Flask, FastAPI, Power BI",Senior,Internship,"Washingtonmouth, SD",90382-110126,2024-04-22,...,-1,-1,6,-1,-1,1995,3,6,2,1904
1996,1997,Roberts-Yu,Automotive,AI Product Manager,"R, Flask, Excel, C++, CUDA, Scikit-learn",Mid,Remote,"Joshuafort, ZA",47848-137195,2023-12-02,...,13,13,0,0,2,1996,2,3,4,1905
1997,1998,"Brooks, Williams and Randolph",Education,Data Analyst,"Hugging Face, Excel, Scikit-learn, R, MLflow",Entry,Contract,"West Brittanyburgh, CG",134994-180108,2023-10-29,...,19,7,4,-1,-1,1997,1,0,5,1906
1998,1999,Castaneda-Smith,Education,Quant Researcher,"AWS, Python, Scikit-learn",Senior,Contract,"Anthonyshire, OM",62388-82539,2024-08-10,...,-1,-1,6,4,3,1998,1,2,5,1907


In [9]:
#df= df.drop(columns=['floc'], axis=1)
inputs = torch.tensor(df.iloc[:,-12:].values, dtype=torch.float32)
df.iloc[:,-13:]



,fskill_1,fskill_2,fskill_3,fskill_4,fskill_5,ftool_1,ftool_2,ftool_3,floc,ftype,ftitle,findustry,fname
0,0,0,0,0,0,0,0,-1,0,0,0,0,0
1,1,1,1,1,-1,1,1,0,1,0,1,1,1
2,2,2,2,2,1,2,2,1,2,0,2,1,2
3,1,3,3,3,2,3,3,2,3,0,3,1,3
4,3,4,1,4,-1,4,0,-1,4,1,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,16,2,14,-1,-1,6,-1,-1,1995,3,6,2,1904
1996,11,6,20,13,13,0,0,2,1996,2,3,4,1905
1997,9,5,4,19,7,4,-1,-1,1997,1,0,5,1906
1998,10,16,4,-1,-1,6,4,3,1998,1,2,5,1907


In [10]:

targets =  torch.tensor(df['Average_Salary'].values, dtype=torch.int64)

targets
max_target_value = max(targets) 
print(f"Maximum target value found: {max_target_value}")

Maximum target value found: 19


## Train/Test Split and Data Preparation


In [11]:
from torch.utils.data import random_split

# Ensure dataset is defined before splitting
if 'dataset' not in locals():
    dataset = TensorDataset(inputs, targets)

torch.manual_seed(42)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Training samples: 1600
Test samples: 400


## Model Definitions


In [12]:
dataset =  TensorDataset(inputs, targets)

In [13]:
dataset[0]
for i in range (1999,2000):
    print(dataset[i])

(tensor([ 2.0000e+01,  1.4000e+01, -1.0000e+00, -1.0000e+00,  6.0000e+00,
        -1.0000e+00, -1.0000e+00,  1.9990e+03,  0.0000e+00,  2.0000e+00,
         2.0000e+00,  1.9080e+03]), tensor(7))


In [14]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [15]:
torch.manual_seed(0)
next(iter(dataloader))

[tensor([[ 1.1000e+01,  6.0000e+00, -1.0000e+00, -1.0000e+00,  7.0000e+00,
           3.0000e+00,  2.0000e+00,  1.2260e+03,  2.0000e+00,  2.0000e+00,
           5.0000e+00,  1.1920e+03],
         [ 1.6000e+01,  7.0000e+00,  3.0000e+00, -1.0000e+00,  3.0000e+00,
           2.0000e+00,  5.0000e+00,  5.0900e+02,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  5.0300e+02],
         [ 2.1000e+01,  1.5000e+01,  0.0000e+00, -1.0000e+00,  6.0000e+00,
           4.0000e+00,  5.0000e+00,  1.6310e+03,  0.0000e+00,  1.0000e+00,
           6.0000e+00,  1.5690e+03],
         [ 6.0000e+00,  1.9000e+01, -1.0000e+00, -1.0000e+00,  4.0000e+00,
           5.0000e+00,  0.0000e+00,  1.1320e+03,  1.0000e+00,  6.0000e+00,
           5.0000e+00,  1.1040e+03]]),
 tensor([11,  7, 13,  5])]

In [16]:
class LinearClassifier(nn.Module):
    def __init__(self):
        super(LinearClassifier, self).__init__()
        # Ensure 'num_classes' is set correctly (e.g., to 20 if targets go up to 19)
        self.fc = nn.Linear(12, 21) 

    def forward(self, x):
        # Flatten the input if necessary (e.g. for image data going into a Linear layer)
        x = x.view(x.size(0), -1) 
        x = self.fc(x)
        return x

In [17]:
torchinfo.summary(LinearClassifier())

Layer (type:depth-idx)                   Param #
LinearClassifier                         --
├─Linear: 1-1                            273
Total params: 273
Trainable params: 273
Non-trainable params: 0

In [18]:
torch.manual_seed(0)
x, targets = next(iter(dataloader))
model = LinearClassifier()
y = model(x)
y


tensor([[ -59.7423,  -49.0172, -168.7090,  548.0013,  -78.6240,  179.1344,
         -512.0240, -427.0768,  -67.2675,  245.5301,   35.4256,  -24.8415,
         -483.0251,   -7.2577, -185.6941,   52.7313,   45.6814, -232.1445,
          182.5670,  106.5504,  -81.2133],
        [ -25.6331,  -21.5431,  -73.2184,  229.9208,  -30.6337,   76.6659,
         -212.6618, -178.5481,  -23.5950,  103.3425,    9.6131,   -8.9795,
         -203.8380,   -6.3964,  -73.6068,   23.9997,   18.1385,  -97.0858,
           78.3726,   45.6595,  -33.5861],
        [ -78.6488,  -59.7034, -228.3792,  724.6681, -105.8770,  232.8002,
         -679.0475, -567.4376,  -89.0435,  325.3814,   45.1253,  -32.5037,
         -643.1141,   -8.4843, -245.7030,   76.3249,   55.7987, -303.4497,
          241.6267,  142.7748, -107.9018],
        [ -53.8720,  -43.6234, -153.7887,  504.8871,  -72.2218,  166.7031,
         -476.2452, -397.3593,  -58.8007,  231.4704,   31.4784,  -24.3682,
         -449.6163,   -6.9894, -171.6938,   49

In [19]:
import torch.nn.functional as F
def loss_fn(y, targets): # 'y' is the first parameter name
    loss = F.cross_entropy(y, targets)
    return loss

In [20]:
torch.manual_seed(0)
x, targets = next(iter(dataloader))
model = LinearClassifier()

# 1. Get the model's predictions
y = model(x) 

with torch.no_grad():
    # 2. Pass 'y' (the predictions/logits) into the loss function, not 'x' (the input data)
    loss = loss_fn(y, targets) 
    
print(loss)

tensor(513.1620)


In [21]:
def accuracy(y, targets):
# 1. Convert logits to predicted class indices
    predictions = torch.argmax(y, dim=1)
    
    # 2. Compare predictions with actual targets (boolean tensor)
    correct_predictions = (predictions == targets)
    
    # 3. Count correct predictions using .sum(). 
    # This result is already a single-element LongTensor.
    num_correct_tensor = correct_predictions.sum()
    
    # 4. Calculate accuracy using tensor division.
    # Convert num_correct_tensor to float for the division operation
    acc_tensor = num_correct_tensor.float() / targets.size(0)
    
    return acc_tensor

In [22]:
def train(model, dataloader, epochs:int, lr:float, debug=False):
        # Set the model to training mode (important for layers like BatchNorm, Dropout)
    model.train()

    # Define the Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

  

    for epoch in range(epochs):
        
        running_loss = 0.0
        running_corrects = 0
        total_samples_epoch = 0

        # Iterate over the data in batches
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            
            # --- Training steps for a single batch ---

            # 1. Zero the parameter gradients
            optimizer.zero_grad()

            # 2. Forward pass: compute the model output (logits)
            y_logits = model(inputs)

            # 3. Calculate loss
            loss = loss_fn(y_logits, targets)
            
            # 4. Backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()

            # 5. Optimizer step: update model parameters
            optimizer.step()


In [23]:
torch.manual_seed(0)
model = LinearClassifier()
x_all, targets_all = dataset[:]
y_all = model(x_all)
print("Before training, accuracy =", accuracy(y_all, targets_all))

train(model, dataloader, epochs=4, lr=0.01, debug=False)

y_all = model(x_all)
print("After training, accuracy =", accuracy(y_all, targets_all))

Before training, accuracy = tensor(0.0165)
After training, accuracy = tensor(0.0810)


In [24]:
class MLPClassifier(nn.Module):
    def __init__(self):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(12, 64)  # First hidden layer
        self.relu = nn.ReLU()         # Activation function
        self.fc2 = nn.Linear(64, 20)  # Output layer
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [25]:
torch.manual_seed(0)
x, targets = next(iter(dataloader))
model = MLPClassifier()
y = model(x)
y

tensor([[  99.0270, -264.9251,  -78.7081,  -41.6630, -368.4966, -188.7650,
          222.4255,  -42.2554,  281.1672, -125.4959,  -99.6009,  -63.3500,
           54.0790,  178.8735,  120.8065,  -90.7125,    9.2846,   39.5906,
           -8.2348,  -63.6293],
        [  41.7060, -109.9044,  -32.4898,  -17.4124, -155.0781,  -79.4910,
           93.2771,  -18.1003,  116.8567,  -51.7646,  -40.8481,  -25.7624,
           22.6626,   73.9056,   50.5655,  -37.6287,    3.2683,   14.8505,
           -4.2315,  -26.0905],
        [ 131.0964, -349.7201, -104.8095,  -54.8540, -487.9455, -248.6155,
          296.1512,  -56.4371,  369.9087, -165.6676, -130.9441,  -84.7414,
           71.9996,  234.6294,  159.4679, -119.5036,   12.8430,   51.0185,
          -13.1759,  -83.4986],
        [  91.4159, -246.1701,  -72.8085,  -38.1553, -341.1308, -175.2783,
          206.8385,  -39.0762,  260.2218, -116.6826,  -90.2290,  -58.9022,
           48.9520,  166.8635,  109.9879,  -82.5993,   10.1184,   34.2917,
    

In [26]:
torch.manual_seed(0)
x, targets = next(iter(dataloader))
model = MLPClassifier()

# 1. Get the model's predictions
y = model(x) 

with torch.no_grad():
    # 2. Pass 'y' (the predictions/logits) into the loss function, not 'x' (the input data)
    loss = loss_fn(y, targets) 
    
print(loss)

tensor(262.5634)


In [27]:

torch.manual_seed(0)
model = MLPClassifier()
x_all, targets_all = dataset[:]
y_all = model(x_all)
print("Before training, accuracy =", accuracy(y_all, targets_all))

train(model, dataloader, epochs=25, lr=0.001, debug=False)

y_all = model(x_all)
print("After training, accuracy =", accuracy(y_all, targets_all))

Before training, accuracy = tensor(0.0345)
After training, accuracy = tensor(0.0975)


## Model Training and Evaluation


In [28]:
def evaluate(model, dataloader):
    model.eval()
    total_correct = 0
    total_samples = 0
    total_loss = 0.0
    
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            total_loss += loss.item()
            
            predictions = torch.argmax(outputs, dim=1)
            total_correct += (predictions == targets).sum().item()
            total_samples += targets.size(0)
    
    accuracy = total_correct / total_samples
    avg_loss = total_loss / len(dataloader)
    return accuracy, avg_loss


In [29]:
class ImprovedMLPClassifier(nn.Module):
    def __init__(self):
        super(ImprovedMLPClassifier, self).__init__()
        self.fc1 = nn.Linear(12, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2) # Add dropout here
        self.fc2 = nn.Linear(64, 20)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x) # Apply dropout after activation
        x = self.fc2(x)
        return x

In [30]:
torch.manual_seed(0)
x, targets = next(iter(dataloader))
model = ImprovedMLPClassifier()
y = model(x)
y

tensor([[  61.5259, -370.0042, -145.8546, -139.6319, -427.7665, -167.1132,
          210.1354,  -47.5512,  330.1723,  -73.6840,  -97.3892,  -20.1820,
           29.7014,  217.3898,  161.9493,  -71.4465,   46.2929,   12.8655,
          -12.4952,  -45.9618],
        [  84.6328, -111.6147,  -36.2852,  -15.5082, -153.7609, -125.5659,
          146.2592,   -8.9250,  131.7863, -103.0458,  -41.1669,   -8.3564,
           43.1857,   75.9433,   38.1018,  -81.3519,   18.2981,    5.9289,
           10.9952,  -33.7652],
        [ 224.5072, -343.8880, -107.6429, -170.5603, -518.9375, -136.2642,
          436.4311,  -15.3075,  388.2523, -241.6260,  -91.2094,  -51.1123,
           22.3146,  135.6509,  222.6239, -145.2629,  178.6929,   61.9865,
         -134.3541,  -62.6938],
        [  98.3006, -234.0290, -120.0708,   -7.2057, -306.2816, -220.2603,
          286.0696,   71.6405,  297.0327, -109.8853, -139.6640, -128.6961,
            3.8185,  242.8548,   89.3725,  -71.4054,  -57.6038,  -21.2307,
    

In [31]:
torch.manual_seed(0)
x, targets = next(iter(dataloader))
model = ImprovedMLPClassifier()

# 1. Get the model's predictions
y = model(x) 

with torch.no_grad():
    # 2. Pass 'y' (the predictions/logits) into the loss function, not 'x' (the input data)
    loss = loss_fn(y, targets) 
    
print(loss)

tensor(330.9029)


In [32]:
torch.manual_seed(0)
model = ImprovedMLPClassifier()
x_all, targets_all = dataset[:]
y_all = model(x_all)
print("Before training, accuracy =", accuracy(y_all, targets_all))

train(model, dataloader, epochs=25, lr=0.001, debug=False)

y_all = model(x_all)
print("After training, accuracy =", accuracy(y_all, targets_all))

Before training, accuracy = tensor(0.0500)
After training, accuracy = tensor(0.1055)


In [33]:
# 1. Set the model to evaluation mode
model.eval() # Ensures layers like Dropout and BatchNorm behave correctly during inference

# 2. Define the target device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # Move the entire model to the selected device

# 3. Prepare the new data and move it to the same device as the model
new_data_list = [[2, 2, 1, 3, 4, 2, 1, 1, 2, 3, 4, 5]]
# Use torch.tensor() directly; it handles data creation and device placement efficiently
new_data_tensor = torch.tensor(new_data_list, dtype=torch.float32).to(device)

# The original line below is redundant and potentially problematic if new_Data was on GPU, as from_numpy expects a CPU array.
# new_data_tensor = torch.from_numpy(new_Data) 

# 4. Perform inference without calculating gradients
with torch.no_grad():
    output = model(new_data_tensor)
    probabilities = torch.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)

# 5. Print results (move tensors to CPU before using with standard Python print/numpy)
print("Outputs (logits):", output.cpu().numpy())
print("Probabilities:", probabilities.cpu().numpy())
print("Predictions:", predicted_class.cpu().numpy())

Outputs (logits): [[-22.009754  -18.896385  -23.526583  -16.499165  -17.883741  -17.972557
   -6.133406   -5.390664   -1.2240237  -5.380385   -5.67304    -0.9911268
   -7.517095   -3.0268734  -4.201011   -8.806344   -2.006874   -2.6195261
   -9.295924  -16.597948 ]]
Probabilities: [[2.9033415e-10 6.5315624e-09 6.3701239e-11 7.1798731e-08 1.7980538e-08
  1.6452448e-08 2.2798604e-03 4.7915634e-03 3.0904809e-01 4.8410683e-03
  3.6127903e-03 3.9009637e-01 5.7145202e-04 5.0939936e-02 1.5744789e-02
  1.5742231e-04 1.4126627e-01 7.6553851e-02 9.6481555e-05 6.5045256e-08]]
Predictions: [11]


In [34]:
import joblib
from sklearn.neural_network import MLPClassifier
# Save the model
filename = 'mlp_model.joblib'
joblib.dump(ImprovedMLPClassifier, filename)

# Load the model later
loaded_clf = joblib.load(filename)

In [35]:
# Load the model from the file
loaded_model = load('mlp_model.joblib')
model.eval()
new_data_list = [[2, 2, 1, 3, 4, 2, 1, 1, 2, 3, 4, 5]]
new_data_tensor = torch.tensor(new_data_list, dtype=torch.float32).to(device)
with torch.no_grad():
    # Run inference here
    output = model(new_data_tensor)
    probabilities = torch.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
print("Outputs (logits):", output.cpu().numpy())
print("Probabilities:", probabilities.cpu().numpy())
print("Predictions:", predicted_class.cpu().numpy())

Outputs (logits): [[-22.009754  -18.896385  -23.526583  -16.499165  -17.883741  -17.972557
   -6.133406   -5.390664   -1.2240237  -5.380385   -5.67304    -0.9911268
   -7.517095   -3.0268734  -4.201011   -8.806344   -2.006874   -2.6195261
   -9.295924  -16.597948 ]]
Probabilities: [[2.9033415e-10 6.5315624e-09 6.3701239e-11 7.1798731e-08 1.7980538e-08
  1.6452448e-08 2.2798604e-03 4.7915634e-03 3.0904809e-01 4.8410683e-03
  3.6127903e-03 3.9009637e-01 5.7145202e-04 5.0939936e-02 1.5744789e-02
  1.5742231e-04 1.4126627e-01 7.6553851e-02 9.6481555e-05 6.5045256e-08]]
Predictions: [11]


## Final Model Training with Train/Test Split


In [36]:
torch.manual_seed(0)
final_model = ImprovedMLPClassifier()

print("Training final model on training set...")
train(final_model, train_loader, epochs=25, lr=0.001)

train_acc, train_loss = evaluate(final_model, train_loader)
test_acc, test_loss = evaluate(final_model, test_loader)

print(f"\nTraining Accuracy: {train_acc:.4f}, Training Loss: {train_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

torch.save(final_model.state_dict(), 'model_state.pth')
print("\nModel saved to 'model_state.pth'")


Training final model on training set...

Training Accuracy: 0.1187, Training Loss: 2.5611
Test Accuracy: 0.0625, Test Loss: 2.7225

Model saved to 'model_state.pth'


## Save Encoding Mappings for Deployment


In [37]:
import pickle

encoding_mappings = {
    'Skill_1': uniqueValues1,
    'Skill_2': uniqueValues2,
    'Skill_3': uniqueValues3,
    'Skill_4': uniqueValues4,
    'Skill_5': uniqueValues5,
    'Tools_1': uniqueValues6,
    'Tools_2': uniqueValues7,
    'Tools_3': uniqueValues8,
    'location': uniqueValues9,
    'employment_type': uniqueValues10,
    'job_title': uniqueValues11,
    'industry': uniqueValues12,
    'company_name': uniqueValues13,
    'salary_categories': targetunique
}

with open('encoding_mappings.pkl', 'wb') as f:
    pickle.dump(encoding_mappings, f)

print("Encoding mappings saved to 'encoding_mappings.pkl'")


Encoding mappings saved to 'encoding_mappings.pkl'


## Load Saved Model and Run Inference


In [38]:
loaded_model = ImprovedMLPClassifier()
loaded_model.load_state_dict(torch.load('model_state.pth'))
loaded_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

new_data_list = [[2, 2, 1, 3, 4, 2, 1, 1, 2, 3, 4, 5]]
new_data_tensor = torch.tensor(new_data_list, dtype=torch.float32).to(device)

with torch.no_grad():
    output = loaded_model(new_data_tensor)
    probabilities = torch.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)

print("Outputs (logits):", output.cpu().numpy())
print("Probabilities:", probabilities.cpu().numpy())
print("Predicted class:", predicted_class.cpu().numpy()[0])


Outputs (logits): [[-19.61344    -14.70145    -23.44898    -13.335011   -12.872398
  -10.950261    -7.0179043   -4.2857237   -1.5769364   -5.685136
   -7.2184453   -1.0357007   -6.2519746   -2.2894762   -2.1961086
   -6.9646926   -1.8118161   -0.40434313 -11.551566   -12.089004  ]]
Probabilities: [[1.8651258e-09 2.5348831e-07 4.0267435e-11 9.9402030e-07 1.5787201e-06
  1.0791434e-05 5.5065536e-04 8.4615666e-03 1.2701710e-01 2.0878236e-03
  4.5059458e-04 2.1823172e-01 1.1844558e-03 6.2288813e-02 6.8384729e-02
  5.8075023e-04 1.0042809e-01 4.1031080e-01 5.9147410e-06 3.4556488e-06]]
Predicted class: 17
